In [70]:
from scipy import stats
import pandas as pd
import os
import numpy as np
from statsmodels.stats.power import TTestIndPower
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import pingouin as pg
from pingouin import power_corr
import seaborn as sns


from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from openpyxl.drawing.image import Image

In [71]:
#df = pd.read_csv('DailyFeatures-7Ps.csv')
df = pd.read_csv('DailyFeatures - wrtOHS&OKS.csv',index_col=0)
df.head()

,participant,Type of fracture,timestamp,position-count,position-duration,position-maximum-distance,motion-sum,motion-ratio,motion-mean,motion-max,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
index,,,,,,,,,,,,,,,,,,,,,
1,1,hip,2022-03-02,7.0,27.0,3.0,NaN,NaN,NaN,NaN,...,3,4,4,4,4,4,38,19.0,11.0,0
2,1,hip,2022-03-03,3.0,13.0,3.0,154.0,0.58,15.40,29.0,...,3,4,4,4,4,4,38,19.0,11.0,0
3,1,hip,2022-03-04,3.0,0.0,3.0,20.0,0.25,6.67,11.0,...,3,4,4,4,4,4,38,19.0,11.0,0
4,1,hip,2022-03-05,0.0,0.0,0.0,153.0,0.67,15.30,32.0,...,3,4,4,4,4,4,38,19.0,11.0,0
5,1,hip,2022-03-06,0.0,0.0,0.0,126.0,0.58,9.69,21.0,...,3,4,4,4,4,4,38,19.0,11.0,0


In [72]:
df = df.drop(['timestamp','Type of fracture'], axis=1)
df.head()

,participant,position-count,position-duration,position-maximum-distance,motion-sum,motion-ratio,motion-mean,motion-max,motion-max-timestamp,heartrate-min,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
index,,,,,,,,,,,,,,,,,,,,,
1,1,7.0,27.0,3.0,NaN,NaN,NaN,NaN,NaN,72.50,...,3,4,4,4,4,4,38,19.0,11.0,0
2,1,3.0,13.0,3.0,154.0,0.58,15.40,29.0,12.0,78.98,...,3,4,4,4,4,4,38,19.0,11.0,0
3,1,3.0,0.0,3.0,20.0,0.25,6.67,11.0,11.0,80.33,...,3,4,4,4,4,4,38,19.0,11.0,0
4,1,0.0,0.0,0.0,153.0,0.67,15.30,32.0,13.0,NaN,...,3,4,4,4,4,4,38,19.0,11.0,0
5,1,0.0,0.0,0.0,126.0,0.58,9.69,21.0,16.0,NaN,...,3,4,4,4,4,4,38,19.0,11.0,0


In [73]:
df1 = df[df['participant'] == 1]
df1 = df1.fillna(df1.mean())

df2 = df[df['participant'] == 2]
df2 = df2.fillna(df2.mean())

df3 = df[df['participant'] == 3]
df3 = df3.fillna(df3.mean())

df4 = df[df['participant'] == 4]
df4 = df4.fillna(df4.mean())

df5= df[df['participant'] == 5]
df5 = df5.fillna(df5.mean())

df6 = df[df['participant'] == 6]
df6 = df6.fillna(df6.mean())

df7 = df[df['participant'] == 7]
df7 = df7.fillna(df7.mean())

df8 = df[df['participant'] == 8]
df8 = df8.fillna(df8.mean())

In [74]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]
df = pd.concat(dfs, ignore_index=True)
df

,participant,position-count,position-duration,position-maximum-distance,motion-sum,motion-ratio,motion-mean,motion-max,motion-max-timestamp,heartrate-min,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
0,1,7.0,27.0,3.0,94.302326,0.517442,9.972791,24.348837,13.348837,72.500000,...,3,4,4,4,4,4,38,19.0,11.0,0
1,1,3.0,13.0,3.0,154.000000,0.580000,15.400000,29.000000,12.000000,78.980000,...,3,4,4,4,4,4,38,19.0,11.0,0
2,1,3.0,0.0,3.0,20.000000,0.250000,6.670000,11.000000,11.000000,80.330000,...,3,4,4,4,4,4,38,19.0,11.0,0
3,1,0.0,0.0,0.0,153.000000,0.670000,15.300000,32.000000,13.000000,76.102453,...,3,4,4,4,4,4,38,19.0,11.0,0
4,1,0.0,0.0,0.0,126.000000,0.580000,9.690000,21.000000,16.000000,76.102453,...,3,4,4,4,4,4,38,19.0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,8,17.0,17.0,0.0,11.000000,0.750000,1.220000,2.000000,12.000000,68.590000,...,5,1,3,1,3,3,31,17.0,12.0,0
428,8,34.0,34.0,1.0,10.000000,0.580000,1.250000,2.000000,13.000000,63.900000,...,5,1,3,1,3,3,31,17.0,12.0,0
429,8,83.0,79.0,0.0,6.000000,0.420000,1.200000,2.000000,11.000000,65.481176,...,5,1,3,1,3,3,31,17.0,12.0,0
430,8,0.0,0.0,0.0,4.000000,0.330000,1.000000,1.000000,9.000000,65.481176,...,5,1,3,1,3,3,31,17.0,12.0,0


In [75]:
data = df.drop(['participant','l2_min'], axis=1)
data.head()

,position-count,position-duration,position-maximum-distance,motion-sum,motion-ratio,motion-mean,motion-max,motion-max-timestamp,heartrate-min,heartrate-max,...,oks-07,oks-08,oks-09,oks-10,oks-11,oks-12,oks,tug,chairstand,sex
0,7.0,27.0,3.0,94.302326,0.517442,9.972791,24.348837,13.348837,72.500000,109.330000,...,3,4,4,4,4,4,38,19.0,11.0,0
1,3.0,13.0,3.0,154.000000,0.580000,15.400000,29.000000,12.000000,78.980000,99.880000,...,3,4,4,4,4,4,38,19.0,11.0,0
2,3.0,0.0,3.0,20.000000,0.250000,6.670000,11.000000,11.000000,80.330000,86.000000,...,3,4,4,4,4,4,38,19.0,11.0,0
3,0.0,0.0,0.0,153.000000,0.670000,15.300000,32.000000,13.000000,76.102453,101.014151,...,3,4,4,4,4,4,38,19.0,11.0,0
4,0.0,0.0,0.0,126.000000,0.580000,9.690000,21.000000,16.000000,76.102453,101.014151,...,3,4,4,4,4,4,38,19.0,11.0,0


In [76]:
# Perform Spearman correlation analysis
#spearman_corr_matrix, _ = spearmanr(data)
spearman_corr_matrix, spearman_p_value = spearmanr(data)

In [77]:
# Create a dataframe from the correlation matrix
corr_df = pd.DataFrame(spearman_corr_matrix, columns=data.columns, index=data.columns)

pvalue_df = pd.DataFrame(spearman_p_value, columns=data.columns, index=data.columns)

## Save the correlation matrix to an Excel file
corr_df.to_excel('CorrMatrix-Daily-AllFractures-wrtOHS&OKS.xlsx')

pvalue_df.to_excel('PMatrix-Daily-AllFractures-wrtOHS&OKS.xlsx')


In [78]:
# Set a threshold for the significance level (alpha)
alpha = 0.0125  # Change this to your desired significance level

# Filter the correlation matrix based on the p-values threshold
filtered_corr_matrix = spearman_corr_matrix.copy()
filtered_corr_matrix[pvalue_df > alpha] = 0

In [79]:
# Filter the features with p-values higher than the threshold
dropped_features = pvalue_df[pvalue_df > alpha].dropna(axis=0, how='all').index.tolist()


In [80]:
dropped_features

['position-count',
 'position-duration',
 'position-maximum-distance',
 'motion-sum',
 'motion-ratio',
 'motion-mean',
 'motion-max',
 'motion-max-timestamp',
 'heartrate-min',
 'heartrate-max',
 'heartrate-mean',
 'heartrate-std',
 'totalsleepduration',
 'deepsleepduration',
 'lightsleepduration',
 'remsleepduration',
 'snoring',
 'durationtosleep',
 'durationtowakeup',
 'wakeupcount',
 'hr_average',
 'hr_min',
 'hr_max',
 'step-sum',
 'step-ratio',
 'step-mean',
 'step-max',
 'step-max-timestamp',
 'Time spent outside (mins)',
 'distance travelled from home',
 'acceleration-sum',
 'l2_mean',
 'l2_std',
 'l2_max',
 'l2_ptp',
 'l2_sum',
 'l2_energy',
 'l2_skewness',
 'l2_kurtosis',
 'l2_peaks',
 'l2_rms',
 'l2_lineintegral',
 'l2_n_above_mean',
 'l2_n_below_mean',
 'l2_n_sign_changes',
 'l2_iqr',
 'l2_iqr_5_95',
 'l2_pct_5',
 'l2_pct_95',
 'l2_entropy',
 'l2_perm_entropy',
 'l2_svd_entropy',
 'sis-01',
 'sis-02',
 'sis-03',
 'sis-04',
 'sis-05',
 'sis-06',
 'sis',
 'SIS (Connectedness)

In [81]:
# Create a dataframe from the filtered correlation matrix
filtered_corr_df = pd.DataFrame(filtered_corr_matrix, columns=data.columns, index=data.columns)
## Save the correlation matrix to an Excel file
filtered_corr_df.to_excel('filt_CorrMatrix-Daily-AllFractures-wrtOHS&OKS.xlsx')

In [82]:
# Save the correlation matrix to an Excel file with heatmap
excel_filename = 'filt_CorrMatrix-Daily-AllFractures-wrtOHS&OKS_heatmap.xlsx'
filtered_corr_df.to_excel(excel_filename, sheet_name='Correlation Matrix')

In [83]:
# Open the Excel workbook and select the active sheet
wb = Workbook()
ws = wb.active

In [84]:
# Create a color map for seaborn heatmap
cmap = sns.color_palette("coolwarm", as_cmap=True)

In [85]:
# Apply color formatting to the Excel cells based on the seaborn heatmap
for row_idx, row in enumerate(filtered_corr_df.index, start=2):
    for col_idx, col in enumerate(filtered_corr_df.columns, start=2):
        cell = ws.cell(row=row_idx, column=col_idx)
        value = filtered_corr_df.at[row, col]
        # Normalize the correlation value to the range [0, 1]
        normalized_value = (value + 1) / 2
        # Map the normalized value to RGB color using the seaborn colormap
        rgb_color = cmap(normalized_value)[:3]
        # Convert RGB to aRGB by adding an alpha channel (255)
        argb_color = (255,) + tuple(int(255 * x) for x in rgb_color)
        # Convert aRGB to HEX
        hex_color = '%02x%02x%02x%02x' % argb_color
        # Apply fill color to the cell
        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")

# Save the modified Excel file
wb.save(excel_filename)

wb.close()

In [86]:
excel_filename2 = 'PMatrix-Daily-AllFractures-wrtOHS&OKS_heatmap.xlsx'
pvalue_df.to_excel(excel_filename2)

In [87]:
# Open the Excel workbook and select the active sheet
wb = Workbook()
ws = wb.active

In [88]:
# Create a color map for seaborn heatmap
cmap = sns.color_palette("coolwarm", as_cmap=True)

In [89]:
# Apply color formatting to the Excel cells based on the seaborn heatmap
for row_idx, row in enumerate(pvalue_df.index, start=2):
    for col_idx, col in enumerate(pvalue_df.columns, start=2):
        cell = ws.cell(row=row_idx, column=col_idx)
        value = pvalue_df.at[row, col]
        # Normalize the correlation value to the range [0, 1]
        normalized_value = (value + 1) / 2
        # Map the normalized value to RGB color using the seaborn colormap
        rgb_color = cmap(normalized_value)[:3]
        # Convert RGB to aRGB by adding an alpha channel (255)
        argb_color = (255,) + tuple(int(255 * x) for x in rgb_color)
        # Convert aRGB to HEX
        hex_color = '%02x%02x%02x%02x' % argb_color
        # Apply fill color to the cell
        cell.fill = PatternFill(start_color=hex_color, end_color=hex_color, fill_type="solid")

# Save the modified Excel file
wb.save(excel_filename2)

In [90]:
# Print the correlation matrix
#print("Spearman Correlation Matrix:")
#print(corr_df)

In [91]:
# Print the correlation matrix
#print("Spearman Pvalue Matrix:")
#print(pvalue_df)

In [92]:
# Plot a heatmap for the correlation matrix using seaborn
#plt.figure(figsize=(10, 8))
#sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
#plt.title('Spearman Correlation Heatmap')
#plt.show()